In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tensorflow.examples.tutorials.mnist import input_data

/home/calin/anaconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
batch_size = 128
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [ ]:
def discriminator(input):
    '''
    input = [28*28, batch_size] array
    return the prob of being real 
    '''
    h0 = tf.layers.dense(input, 28*28, activation = tf.nn.relu, name='d0')
    h1 = tf.layers.dense(h0, 1,  name='d1')
    prob = tf.sigmoid(h1, 'd2')
    
    return prob

def generator(inputs):
    '''
    receives random noise and tries to reproduce mnist image
    '''
    h0 = tf.layers.dense(inputs, 100, activation=tf.nn.relu, name='g0')
    h1 = tf.layers.dense(h0, 28*28, name='g1')
    h2 = tf.sigmoid(h1, name='g2')
    return h2


with tf.variable_scope('G'):
    z = tf.placeholder(tf.float32, shape=[None, 100], name='z')
    fake_samples = generator(z)
   
    
with tf.variable_scope('D') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784], name='inputs')
    D_real = discriminator(x)
    scope.reuse_variables()
    D_fake = discriminator(fake_samples)
    

vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('D/')]
g_params = [v for v in vars if v.name.startswith('G/')]

'''
loss_d = - tf.reduce_mean(tf.log(D1) + tf.log(1.0 - D2))
loss_g = - tf.reduce_mean(tf.log(D2))
''' 

G_loss = -tf.reduce_mean(D_fake)
D_loss = tf.reduce_mean(D_fake) - tf.reduce_mean(D_real)

epsilon = tf.random_uniform(
    shape=[batch_size, 1, 1, 1],
    minval=0.,
    maxval=1.
)

X_hat = X_real + epsilon * (X_fake - X_real)
D_X_hat = D(X_hat, reuse=True)
grad_D_X_hat = tf.gradients(D_X_hat, [X_hat])[0]

red_idx = [-1]

slopes = tf.sqrt(tf.reduce_sum(tf.square(grad_D_X_hat), reduction_indices=red_idx))
gradient_penalty = tf.reduce_mean((slopes - 1.)**2)
D_loss += 10 * gradient_penalty


d_optimizer = tf.train.AdamOptimizer().minimize(loss_d, var_list=d_params)
g_optimizer = tf.train.AdamOptimizer().minimize(loss_g, var_list=g_params)
    

In [ ]:
num_steps = 100000
saver = tf.train.Saver()
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(num_steps):
        
        if i % 1000 == 0:
            save_path = saver.save(sess, "/tmp/model.ckpt")

        
        if i % 100 == 0:
            fake_data = sess.run(fake_samples, feed_dict={z: sample_Z(16, 100)})
            fig = plot(fake_data)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            plt.close(fig)

        
        train_data, _ = mnist.train.next_batch(batch_size)
        samples = sample_Z(batch_size, 100)

        #update discriminator
        loss_disc, _ = sess.run([loss_d, d_optimizer], {
            x: train_data,
            z: samples
        })

        #update generator
        loss_gen, _ = sess.run([loss_g, g_optimizer], {
            z: sample_Z(batch_size, 100)
        })
        
        loss_gen, _ = sess.run([loss_g, g_optimizer], {
            z: sample_Z(batch_size, 100)
        })
        
        if i % 1000 == 0:
            print(loss_disc, loss_gen)
    

In [ ]:
tf.reset_default_graph()

In [ ]:
#vanilla gan
"""
def discriminator(input):
    '''
    input = [28*28, batch_size] array
    return the prob of being real 
    '''
    h0 = tf.layers.dense(input, 28*28, activation = tf.nn.relu, name='d0')
    h1 = tf.layers.dense(h0, 1,  name='d1')
    prob = tf.sigmoid(h1, 'd2')
    
    return prob

def generator(inputs):
    '''
    receives random noise and tries to reproduce mnist image
    '''
    h0 = tf.layers.dense(inputs, 100, activation=tf.nn.relu, name='g0')
    h1 = tf.layers.dense(h0, 28*28, name='g1')
    h2 = tf.sigmoid(h1, name='g2')
    return h2


with tf.variable_scope('G'):
    z = tf.placeholder(tf.float32, shape=[None, 100], name='z')
    fake_samples = generator(z)
   
    
with tf.variable_scope('D') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784], name='inputs')
    D1 = discriminator(x)
    scope.reuse_variables()
    D2 = discriminator(fake_samples)
    

vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('D/')]
g_params = [v for v in vars if v.name.startswith('G/')]

loss_d = - tf.reduce_mean(tf.log(D1) + tf.log(1.0 - D2))
loss_g = - tf.reduce_mean(tf.log(D2))
    
d_optimizer = tf.train.AdamOptimizer().minimize(loss_d, var_list=d_params)
g_optimizer = tf.train.AdamOptimizer().minimize(loss_g, var_list=g_params)
    
"""